In [83]:
import pandas as pd
from math import log2

pd.options.display.float_format = "{:,.0f}".format


## Chord System

## function


In [84]:
class ChordSystem:
    def __init__(self, ring_size, active_nodes):
        self.ring_size = ring_size
        self.finger_table_size = int(log2(self.ring_size))
        self.active_nodes = active_nodes

    def get_succ(self, node):
        i = node
        while i not in self.active_nodes:
            i = (i + 1) % self.ring_size
        return i

    def get_succs(self, nodes=None):
        if not nodes:
            nodes = [i for i in range(self.ring_size)]
        succs = [pd.Series({"succ": self.get_succ(node)}, name=node) for node in nodes]
        return pd.DataFrame(succs)

    def get_finger_table(self, active_node):
        finger_table = {}
        for i in range(self.finger_table_size):
            finger_table[f"i_{i}"] = self.get_succ(
                (active_node + 2**i) % self.ring_size
            )
        return pd.Series(finger_table, name=active_node)

    def get_finger_tables(self, nodes=None):
        if not nodes:
            nodes = self.active_nodes
        return pd.DataFrame(
            [self.get_finger_table(node) for node in nodes]
        ).sort_index()

    def get_path(self, start, end):
        steps_taken = 0
        finish = self.get_succ(end)
        path = f"{start}"
        current = start
        while current != finish:
            steps_taken += 1
            if steps_taken > self.ring_size:
                raise Exception("something wrong "+ path)
            
            finger_table = self.get_finger_table(current).values
            current = finger_table[0]
            for node in finger_table[1:]:
                if ((start > end) and (node <= start) and (node > end)) or (
                    (start < end) and ((node > end) or node <= start)
                ):
                    break
                current = node
                if node == finish:
                    break
            path += f"-{current}"
        return path

    def get_finger_tables_for_path(self, start, end):
        nodes = [int(node) for node in self.get_path(start, end).split("-")]
        return self.get_finger_tables(nodes)


## test

In [85]:
# import pytest
# @pytest.fixture
def test_cs():
    return ChordSystem(16, [2, 4, 7, 8, 10, 12, 15])


test_cs = test_cs()


In [86]:
def test_get_succ(test_cs: ChordSystem):
    assert test_cs.get_succ(0) == 2
    assert test_cs.get_succ(2) == 2
    assert test_cs.get_succ(10) == 10
    assert test_cs.get_succ(15) == 15
    assert test_cs.get_succ(16) == 2


test_get_succ(test_cs)


In [87]:
def test_get_succs(test_cs:ChordSystem):
    expected = [2, 2, 10, 15, 2]
    result = list(test_cs.get_succs([0,2,10,15,16])["succ"])
    assert expected == result 
    
test_get_succs(test_cs)

In [88]:
def test_get_finger_table(test_cs: ChordSystem):
    result = list(test_cs.get_finger_table(2))
    expected = [4, 4, 7, 10]
    assert result == expected
    result = list(test_cs.get_finger_table(15))
    expected = [2, 2, 4, 7]
    assert result == expected


test_get_finger_table(test_cs)


In [89]:
def test_get_finger_tables(test_cs: ChordSystem):

    expected = [10, 7]
    result = list(test_cs.get_finger_tables([2, 15])["i_3"])
    assert expected == result


test_get_finger_tables(test_cs)


In [90]:
def test_get_path(test_cs: ChordSystem):
    result = test_cs.get_path(2, 14)
    expected = "2-10-12-15"
    assert expected == result

    result = test_cs.get_path(12, 9)
    expected = "12-4-8-10"
    assert expected == result

    # Beispiel für Routing über Finger-Tabellen auf Frau Ma's Folien
    cs = ChordSystem(ring_size=16, active_nodes={0, 3, 5, 8, 12, 14})
    result = cs.get_path(3, 15)
    expected = "3-12-14-0"
    assert result == expected
    
    # EA 4 Chord Aufgabe
    cs = ChordSystem(ring_size=16, active_nodes={0, 2, 6, 8, 10, 12})
    result = cs.get_path(2, 9)
    expected = '2-6-8-10'
    assert result == expected


test_get_path(test_cs)


# creation

In [91]:
ring_size = 16
active_nodes = {2, 4, 7, 8, 10, 12, 15}
cs = ChordSystem(ring_size, active_nodes)


# succ

In [92]:
nodes_to_get_succ_for = {0, 2, 4, 6, 13, 15}
cs.get_succs(nodes_to_get_succ_for)


,succ
0,2
2,2
4,4
6,7
13,15
15,15


# finger table

In [93]:
cs.get_finger_tables()

,i_0,i_1,i_2,i_3
2,4,4,7,10
4,7,7,8,12
7,8,10,12,15
8,10,10,12,2
10,12,12,15,2
12,15,15,2,4
15,2,2,4,7


# path

In [94]:
start = 4
end = 11
cs.get_path(start, end)


'4-8-10-12'

# finger tables for path

In [95]:
cs.get_finger_tables_for_path(start,end)

,i_0,i_1,i_2,i_3
4,7,7,8,12
8,10,10,12,2
10,12,12,15,2
12,15,15,2,4
